In [34]:
import json
import pandas as pd
import numpy as np
from pathlib import Path
import collections


from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [35]:
# path = "../error_category_classification/dbdc5_ja_dev_labeled/"
path = "../hand_labeled/"
datalist = ['DCM', 'DIT', 'IRS']

output = "./"

In [36]:
def read_json_with_NoErr(path:str, datalist:list) -> pd.DataFrame:
    cols = ['did', 'tid', 'usr', 'sys', 'ec']
    df = pd.DataFrame(index=[], columns=cols)

    for p in datalist:
        datapath = Path(path + p + '/')
        for file in datapath.glob("*.json"):
            with open(file, "r") as f:
                json_data = json.load(f)
                did = json_data["did"]
                for t in json_data["turns"]:
                    if t["turn-index"] == 0:
                        continue
                    if t["speaker"] == "U":
                        usr = t["utterance"]
                        continue
                    if t["speaker"] == "S" :
                        tid = t["turn-index"]
                        sys = t["utterance"]
                        if t["error_category"]:
                            ec = t["error_category"]
                        else:
                            ec = ["No-Err"]
                        df = df.append(pd.DataFrame([did, tid, usr, sys, ec], index = cols).T)
    df.reset_index(inplace=True, drop=True)
    return df

In [37]:
def extract_error(df, errors):
    utterances = []
    for d, u, s, e in zip(df.did, df.usr, df.sys, df.ec):
        for err in errors:
            if err in e:
                utterances.append([u, s])
    return utterances

In [38]:
def no_error(df):
    utterances = []
    system = []
    user = []
    for d, u, s, e in zip(df.did, df.usr, df.sys, df.ec):
        if e[0] == "No-Err":
            utterances.append([ u, s])
        # system.append(s)
        # user.append(u)
    return utterances

In [39]:
df = read_json_with_NoErr(path, datalist)

In [40]:
df

,did,tid,usr,sys,ec
0,1470646494,2,そうですね。最近とても暑いですから。,海に行かないんですね？,[Contradiction]
1,1470646494,4,休日に行きたいと思います。,海は大丈夫ですね,[Lack of information]
2,1470646494,6,はい。あなたは海に行きますか？,海は危険です,"[Ignore question, Self-contradiction]"
3,1470646494,8,何故ですか？,海はいけないです,[Ignore question]
4,1470646494,10,そうですか。山に行くのはどうでしょうか？,わあ！いいですね！,[No-Err]
...,...,...,...,...,...
1995,1503290726,12,何を別売りしているの？,地下鉄で。名大前ができてすごく便利になりましたね、ここの大学。,[Repetition]
1996,1503290726,14,今は2回押してないよ,紅葉も遅いみたいですしね。私は、いつもベーッシクな色で変わりばえしないのですが。,[Topic transition error]
1997,1503290726,16,ベーシックだよね,何処にあるお店ですか？ベースコートもトップコートも塗ると持ちが違いますよ。,[Topic transition error]
1998,1503290726,18,君は女性なんだね,中間ですか？,[Lack of information]


In [41]:
errors = ["Topic transition error", "Unclear intention", "Lack of information"]
errors = ["Lack of information"]
errors = ["Topic transition error"]

In [42]:
ut = extract_error(df, errors)

In [43]:
ut_no = no_error(df)

In [44]:
print(len(ut))
print(len(ut_no))
print(ut_no[0])

192
651
['そうですか。山に行くのはどうでしょうか？', 'わあ！いいですね！']


In [45]:
import spacy
nlp = spacy.load('ja_ginza')
vec_dim = nlp("形態素").vector.shape
vec_dim

(300,)

In [46]:
from pyknp import Juman
from sentence_transformers import SentenceTransformer
import scipy.spatial
model_path = "/home/yamada/Downloads/training_bert_japanese"
Nmodel = SentenceTransformer(model_path, show_progress_bar=False)
emb_dim = Nmodel.encode(["お辞儀をしている男性会社員"])[0].shape[0]

In [47]:
def _in(arg1, arg2):
    result = False
    if isinstance(arg1, list):
        for element in arg1:
            if isinstance(element, str):
                if element in arg2:
                    result = True
                    break
            else:
                if element == arg2:
                    result = True
                    break
    else:
        if isinstance(arg1, str):
            if arg1 in arg2:
                result = True
        else:
            if arg1 == arg2:
                result = True
    return result

In [48]:
def filter_pos_vector(pos_list, text):
    vec = np.zeros(vec_dim)
    doc = nlp(text, disable=['ner'])
    for token in doc:
        # print(token, token.tag_)
        if "助動詞" not in token.tag_ and _in(pos_list, token.tag_):
            # print(token, token.tag_)
            vec += token.vector
    return vec

In [72]:
def feature_extraction(pos_list, convs, mode, is_norm=False):
    feature_list = []
    for utt in convs:

        if mode == "dif":
            vector = Nmodel.encode(utt)
            # usr_vec = filter_pos_vector(pos_list, utt[0] )
            usr_vec = vector[0]
            # sys_vec = filter_pos_vector(pos_list, utt[1] )
            sys_vec = vector[1]
            dif_vec = usr_vec - sys_vec

            # feature_list.append( dif_vec/np.linalg.norm(dif_vec) )
            # feature_list.append( dif_vec )
            if np.linalg.norm(dif_vec)==0:
                print(utt)
            if is_norm:
                # print(np.linalg.norm(dif_vec))
                if np.linalg.norm(dif_vec)==0:
                    vec = np.concatenate( [dif_vec, dif_vec/0.05] )
                else:
                    vec = np.concatenate( [dif_vec, dif_vec/np.linalg.norm(dif_vec)] )
                feature_list.append(vec)
            else:
                feature_list.append(dif_vec)
    
    return np.array(feature_list, dtype="float32")


In [50]:
def remove_norm_zero(pos_list, convs):
    convs_removed = []
    for utt in convs:
        usr_vec = filter_pos_vector(pos_list, utt[0] )
        sys_vec = filter_pos_vector(pos_list, utt[1] )
        dif_vec = usr_vec - sys_vec
        if np.linalg.norm(dif_vec)==0 or np.linalg.norm(usr_vec)==0 or np.linalg.norm(sys_vec)==0:
            continue
        convs_removed.append(utt)
    return convs_removed

In [51]:
pos_list = ["名詞", "動詞", "形状詞", "形容詞"]
ut_new = remove_norm_zero(pos_list, ut)
ut_no_new = remove_norm_zero(pos_list, ut_no)

In [52]:
import random
def fake_error(convs, times):
    user_list = []
    system_list = []
    for utt in convs:
        user_list.append(utt[0])
        system_list.append(utt[1])
    
    fake_convs = []
    for _ in range(times):
        fake_convs.append( [random.choice(user_list), random.choice(system_list)] )
    return fake_convs

In [53]:
fake_error(ut_no_new, len(ut_no_new)-len(ut_new))

[['よろしくお願いします。', '日本語で考えて、英語で話していると相手にはどうも伝わりにくいようですので、英単語を並べたほうがまだ伝わるようですよ。'],
 ['好きな食べ物はありますか？', '扇風機を併用するんですね。エアコンは欲しいですね'],
 ['こんにちは。いつも家では仕事をしています。',
  'ウィザーディング・ワールド・オブ・ハリー・ポッターエリアでもユニバーサル・スタジオ・ジャパンのようかい体操第一のオープン前から、たくさんコマーシャルが流れていました。'],
 ['どんな食べ物にも合いますね',
  '私もどちらかというとインドア派で家でのんびり映画とか見てるのが好きなんですが、１３さんは、家でどんな風にのんびり過ごすのが好きですか？'],
 ['そんな事もないと思いますよ。', 'ご想像にお任せします。'],
 ['最近はまってることは読書です', '最近はさっぱりですね。'],
 ['大人の味ってほろ苦いというイメージですかね。', '面白いですよね。よく小説とか読まれたりします？'],
 ['栗原はるみさんですね', 'ミートソースは好きですか。子供から大人まで好きな食べ物だね。'],
 ['はい。麺類は美味しいから好きです。', '腰は痛みが走るんですよねぇ'],
 ['その通り。基本です。', '海水浴は楽しいんですか？？海は楽しいですね'],
 ['日焼けすると赤くなる', '確かに、最近は昔嫌いだったものが好きになってきました。'],
 ['コブクロと言えば、絢香とのコラボの曲が好きです。', 'インターネットは便利ですね'],
 ['ボードゲームです', '麺類は食べやすいですね'],
 ['忙しくなりそうですね。専攻は？', '予防を怠るようです'],
 ['私も沖縄大好きですよ。', '防水はいいそうです'],
 ['何が好きになったんでしょう。私はコーヒーが飲めるようになりましたよ。', '防水はいいそうです'],
 ['占いというのは下らないね',
  '色々あって難しいところですが、最近だと我が名は海師・王様の仕立屋・コンシェルジュなどが気に入ったので１巻からまとめ買いしたところです。'],
 ['そこも興味があります。食べてみたいな。', '気持ちがいいですね'],
 ['晴れと雨ではどちらが好きですか', 'マ

In [54]:
UT = ut_new + fake_error(ut_no_new, len(ut_no_new)-len(ut_new))
UT_NO = ut_no_new


In [55]:
print("UT: {0}".format(len(UT)))
print("UT_NO: {0}".format(len(UT_NO)))

UT: 528
UT_NO: 528


In [73]:
X = feature_extraction(pos_list, UT + UT_NO , mode="dif", is_norm=True)
y = np.concatenate( [ np.ones(len(UT)), np.zeros(len(UT_NO)) -1 ] )
y

array([ 1.,  1.,  1., ..., -1., -1., -1.])

In [57]:
print("ut: {0}, ut_new: {1}, dif:{2}".format(len(ut), len(ut_new), len(ut)- len(ut_new)))
print("ut_no: {0}, ut_no_new: {1}, dif:{2}".format(len(ut_no), len(ut_no_new), len(ut_no)- len(ut_no_new)))

ut: 192, ut_new: 170, dif:22
ut_no: 651, ut_no_new: 528, dif:123


In [74]:
model = SVC(kernel="rbf", random_state=2, C=0.1)


In [75]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [76]:
model.fit(X_train, y_train)

SVC(C=0.1, random_state=2)

In [77]:
y_pred = model.predict(X_test)
# y_pred = np.ones(len(y_test))-2

In [78]:
len(y_test)

212

In [79]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
print('confusion matrix = \n', confusion_matrix(y_true=y_test, y_pred=y_pred))
print('accuracy = ', accuracy_score(y_true=y_test, y_pred=y_pred))
print('precision = ', precision_score(y_true=y_test, y_pred=y_pred))
print('recall = ', recall_score(y_true=y_test, y_pred=y_pred))
print('f1 score = ', f1_score(y_true=y_test, y_pred=y_pred))

confusion matrix = 
 [[ 35  67]
 [  2 108]]
accuracy =  0.6745283018867925
precision =  0.6171428571428571
recall =  0.9818181818181818
f1 score =  0.7578947368421052


In [75]:
for x in ut_no[:10]:
    
    f = feature_extraction(pos_list, x, mode="dif", is_norm=True)
    pred = np.argmax( clf.predict(f), axis=0 )
    print(pred, x)

0 ['そうですか。山に行くのはどうでしょうか？', 'わあ！いいですね！']
0 ['明日はとても暑くなるみたいですね。', 'ご存知になりたいです']
0 ['涼しくなってきたら、一緒に山へ行きたいですね。', 'ご存知ですいますよねー']
0 ['美味しいですね。', '美味しいよ']
0 ['でも高いですね', 'いくら']
1 ['サスペンスが好きです', '映画はヒューマン系が好きですね']
0 ['そうですか', 'あはい']
0 ['こんばんは。今日は暑かったですね。', 'ところで、スイカの種って実は栄養豊富なんだそうですよ。']
0 ['ありがとう。野菜もたっぷり食べて、栄養をバランスよく摂りたいです。', '野菜でヘルシーよねー']
0 ['こんにちは。元気ですね', '夕食は食べましたか？']


In [80]:
from sklearn.ensemble import AdaBoostClassifier
clf = AdaBoostClassifier()

In [81]:
clf.fit(X_train, y_train)

AdaBoostClassifier()

In [82]:
y_pred = clf.predict(X_test)
print('confusion matrix = \n', confusion_matrix(y_true=y_test, y_pred=y_pred))
print('accuracy = ', accuracy_score(y_true=y_test, y_pred=y_pred))
print('precision = ', precision_score(y_true=y_test, y_pred=y_pred))
print('recall = ', recall_score(y_true=y_test, y_pred=y_pred))
print('f1 score = ', f1_score(y_true=y_test, y_pred=y_pred))

confusion matrix = 
 [[57 45]
 [39 71]]
accuracy =  0.6037735849056604
precision =  0.6120689655172413
recall =  0.6454545454545455
f1 score =  0.6283185840707964


['test_sen.ipynb',
 '.gitignore',
 'LiveComp',
 'paper',
 'UNI',
 '.git',
 'theme',
 'corpus',
 'test_bert.py',
 'sentece.py',
 'path_test.py']